# <center>Machine Learning in Python</center>  

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

train_df = pd.read_csv('C:/Users/kashif/Desktop/kc_workshop/final/keras/dataset/data/train_data.csv')
test_df = pd.read_csv('C:/Users/kashif/Desktop/kc_workshop/final/keras/dataset/data/test_data.csv')
train_df['target'] = train_df['target'].astype(str)
train_df.head()

In [ ]:
src_path_train = "C:/Users/kashif/Desktop/kc_workshop/final/keras/dataset/data/train/"
src_path_test = "C:/Users/kashif/Desktop/kc_workshop/final/keras/dataset/data/test/"

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale=1 / 255.0)

batch_size = 4
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=src_path_train,
    x_col="img_code",
    y_col="target",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=src_path_train,
    x_col="img_code",
    y_col="target",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=42
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=src_path_test,
    x_col="img_code",
    target_size=(100, 100),
    batch_size=1,
    class_mode=None,
    shuffle=False,
)

In [ ]:
def simple_lr():
    # initialize model
    model = Sequential()
    # flatten or vectorize input layer images
    model.add(Flatten(input_shape=(img_height,img_width,3)))
    # add an input layer and output layer
    model.add(Dense(1, input_dim=img_height*img_width, activation='sigmoid'))   
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
    return model
model = prepare_model()
model.fit_generator(train_generator,
                    validation_data = train_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_steps = valid_generator.n//valid_generator.batch_size,
                    epochs=5)


In [ ]:
score = model.evaluate_generator(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predict=model.predict_generator(test_generator, steps = len(test_generator.filenames))
predict[:5]

y_classes = predict.argmax(axis=-1)
print(y_classes)